# Making some test runs and setting observation values and weights

In the previous notebook, we constructed a (very) high-dimensional PEST(++) around the structured-grid MF6 model.  In this notebook, we will verify that interface by running a few test runs.  Then we will set the actual observation values and associated weights in the control file in preparation for some more advanced analyses. 

In [ ]:
import sys
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
sys.path.append('../../dependencies/')
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
font = {'size'   : 12}
mpl.rc('font', **font)
import flopy as fp
import pyemu


In [ ]:
mname = "sgn_50"
t_d = os.path.join("..","..","models","template")
assert os.path.exists(t_d)

In [ ]:
[f for f in  os.listdir(t_d) if f[-3:] in ["pst","tpl","ins"]]

Sweet!  we see all of our pest interface files...let's load the control file

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"sgn.pst"))

Kewl - now lets test a single run of the process, just to make sure everything is working as expected...

In [ ]:
pst.control_data.noptmax = 0
pst.observation_data.loc[:,"weight"] = 1.0
pst.write(os.path.join(t_d,"sgn.pst"),version=2)

We will use a system call utility in pyemu that is operating system agnostic:

In [ ]:
pyemu.os_utils.run("pestpp-ies sgn.pst",cwd=t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"sgn.pst"))
pst.phi

Nice! Since we have not adjusted the observation data or parameters, and the observation values in the control file are just the values in the existing output files, we expect the objective function value to be at or very near 0.0. Let's run the mean parameter values in the prior parameter ensemble - this is done with `noptmax=-2`

In [ ]:
pst.control_data.noptmax = -2
# tell pestpp-ies to use our prior parameter ensemble
pst.pestpp_options["ies_par_en"] = "prior.jcb"
pst.write(os.path.join(t_d,"sgn.pst"),version=2)
pyemu.os_utils.run("pestpp-ies sgn.pst",cwd=t_d)

Now, lets run a single stochastic parameter realization, just to see how that works...

In [ ]:
pe = pyemu.ParameterEnsemble.from_binary(pst=pst,filename=os.path.join(t_d,"prior.jcb"))
pst.parameter_data.loc[:,"parval1"] = pe.loc[pe.index[0],pst.par_names].values
pst.control_data.noptmax = 0
pst.write(os.path.join(t_d,"sgn_test.pst"),version=2)
pyemu.os_utils.run("pestpp-ies sgn_test.pst",cwd=t_d)
tpst = pyemu.Pst(os.path.join(t_d,"sgn_test.pst"))
tpst.phi

Ok, now the phi is higher, as expected.  Let's visualize the MF6 HK input array for this realization:

Here we load the dataframe that informs the multiplier parameter process that pyemu uses are runtime:

In [ ]:
df = pd.read_csv(os.path.join(t_d,"mult2model_info.csv"),index_col=0)
df.head()

Let's get all parameterization files related to HK in model layer 1:

In [ ]:
lay1_hk = df.loc[df.model_file==mname+".npf_k_layer1.txt",:]
lay1_hk

Now load the original (e.g. existing) model layer 1 HK and the resulting model layer 1 HK array that MF6 will see (after all the multiplier arrays have been applied)

In [ ]:
org_arr = np.loadtxt(os.path.join(t_d,lay1_hk.org_file.iloc[0]))
final_arr = np.loadtxt(os.path.join(t_d,lay1_hk.model_file.iloc[0]))

In [ ]:
fig,axes = plt.subplots(1,lay1_hk.shape[0]+2,figsize=(8*lay1_hk.shape[0]+2,8))
axes[0].imshow(org_arr,vmin=final_arr.min(),vmax=final_arr.max())
axes[0].set_title("original HK array")
axes[-1].imshow(final_arr,vmin=final_arr.min(),vmax=final_arr.max())
axes[-1].set_title("final HK array for MF6")
for i,mlt_file in enumerate(lay1_hk.mlt_file.values):
    arr = np.loadtxt(os.path.join(t_d,mlt_file))
    axes[i+1].imshow(arr)
    axes[i+1].set_title(mlt_file.split(".")[0].split("_")[-1])

From left to right, we can visualize the multiplier parameter process that we are using.

# Setting observation values and weights

This is always painful!.  So we are gonna load up the control file and the hob file we found floating around.  Then we are gonna assign the `obsval` quantities to the observations in the control file that correspond to the hob observed quantities.  

In [ ]:
hob = pd.read_csv("gv39.hob",delim_whitespace=True,skiprows=4,header=None,names=["site","l","r","c","obsval"],usecols=[0,1,2,3,8])
hob.site = hob.site.str.lower()
hob.index = hob.site
obs = pst.observation_data
obs.loc[:,"weight"] = 0.0
hobs = obs.loc[obs.obsnme.str.contains("head"),:]
hobs.loc[:,"time"] = hobs.time.astype(float)
# adding the "_time" suffix causes us to only use layer 1 obs...
for site,obsval in zip(hob.site,hob.obsval):
    print(site)
    hobs_site = hobs.loc[hobs.obsnme.str.contains(site)]
    #assert hobs_site.shape[0] > 0,site
    if hobs_site.shape[0] == 0:
        print("missing",site)
        continue
    hobs_site.sort_values(by="time",inplace=True)
    obs.loc[hobs_site.obsnme.iloc[-1],"obsval"] = obsval
    obs.loc[hobs_site.obsnme.iloc[-1],"weight"] = 3.0
    obs.loc[hobs_site.obsnme.iloc[-1],"obgnme"] += "_measured"    
    
    
    
# hob.loc[:,"obsnme"] = hob.apply(lambda x: [o for o in obs.obsnme if x.site+"_time" in o],axis=1)
# hob.obsnme.apply(lambda x: x.sort())
# print(hob.obsnme.values)
# print(hob.loc[hob.obsnme.apply(lambda x: len(x) > 4),"obsnme"].values)
# hob.loc[:,"obsnme"] = hob.obsnme.apply(lambda x: x[0] if len(x)==1 else np.NaN)
# hob.dropna(inplace=True)
# assert hob.shape[0] > 0
# hob.index = hob.obsnme
# obs.loc[hob.obsnme,"obsval"] = hob.obsval
# obs.loc[hob.obsnme,"weight"] = 3.0
assert len(pst.nnz_obs_names) > 0
pst.nnz_obs_names

Let's also set the obsvals for the concentration obs

In [ ]:
cob = pd.read_csv("pce_obsval.csv")
cob.loc[:,"site"] = cob.site.str.replace("_","")

In [ ]:
cob = cob.loc[cob.pce > 0,:]
cob

In [ ]:
cobs = obs.loc[obs.obgnme.str.contains("conc"),:]
cobs.loc[:,"time"] = cobs.time.astype(float)
for site,obsval in zip(cob.site,cob.pce):
    cobs_site = cobs.loc[cobs.usecol==site,:]
    assert cobs_site.shape[0] > 0,site
    cobs_site.sort_values(by="time",inplace=True)
    #print(cobs_site)
    #break
    obs.loc[cobs_site.obsnme.iloc[-1],"obsval"] = obsval
    obs.loc[cobs_site.obsnme.iloc[-1],"weight"] = 1.0
    obs.loc[cobs_site.obsnme.iloc[-1],"obgnme"] += "_measured"

OK! now we can save this control file and have some fun!

In [ ]:
pst.control_data.noptmax = -1
pst.write(os.path.join(t_d,"sgn.pst"),version=2)